##구글 BERT의 마스크드 언어모델(Masked Language Model)실습

사전 학습된 한국어 BERT 를 이용하여 마스크드 언어 모델을 실습해봅시다. 이번 실습을 위해서만이 아
니라 앞으로 사전 학습된 BERT 를 사용할 때는 transformers 라는 패키지를 자주 사용하게 됩니다. 실습
환경에 transformers 패키지를 설치해둡시다.

In [1]:
!pip install transformers

### 1. 마스크드 언어 모델과 토크나이저

transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT 는 이미 누군가가 학습해둔 모
델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다. 예를 들어서
A 라는 이름의 BERT 를 사용하는데, B 라는 이름의 BERT 의 토크나이저를 사용하면 모델은 텍스트를 제
대로 이해할 수 없습니다. A 라는 BERT 의 토크나이저는’ 사과’ 라는 단어를 36 번으로 정수 인코딩하는
반면에, B 라는 BERT 의 토크나이저는’ 사과’ 라는 단어를 42 번으로 정수 인코딩하는 등 단어와 맵핑되는
정수 정보 자체가 다르기 때문입니다.

In [2]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

TFBertForMaskedLM.from_pretrained(‘BERT 모델 이름’) 을 넣으면 [MASK] 라고 되어있는 단어를 맞추
기 위한 마스크드 언어 모델링을 위한 구조로 BERT 를 로드합니다. 다시 말해서 BERT 를 마스크드 언어
모델 형태로 로드합니다.

AutoTokenizer.from_pretrained(‘모델 이름’) 을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크
나이저를 로드합니다.

In [3]:
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

###2. BERT의 입력
’Soccer is a really fun [MASK]’ 라는 임의의 문장이 있다고 해봅시다. 이를 마스크드 언어 모델의 입력으
로 넣으면, 마스크드 언어 모델은 [MASK] 의 위치에 해당하는 단어를 예측합니다. 마스크드 언어 모델의
예측 결과를 보기위해서 bert‐large‐uncased 의 토크나이저를 사용하여 해당 문장을 정수 인코딩해봅시
다.

In [4]:
inputs = tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


토크나이저로 변환된 결과에서 input_ids 를 통해 정수 인코딩 결과를 확인할 수 있습니다.

In [5]:
print(inputs['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


토크나이저로 변환된 결과에서 token_type_ids 를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 확
인할 수 있습니다.

In [6]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


현재의 입력은 문장이 두 개가 아니라 한 개이므로 여기서는 문장 길이만큼의 0 시퀀스를 얻습니다. 만약
문장이 두 개였다면 두번째 문장이 시작되는 구간부터는 1 의 시퀀스가 나오게 되지만, 여기서는 해당되
지 않습니다.
토크나이저로 변환된 결과에서 attention_mask 를 통해서 실제 단어와 패딩 토큰을 구분하는 용도인 어
텐션 마스크를 확인할 수 있습니다.

In [7]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


현재의 입력에서는 패딩이 없으므로 여기서는 문장 길이만큼의 1 시퀀스를 얻습니다. 만약 뒤에 패딩이
있었다면 패딩이 시작되는 구간부터는 0 의 시퀀스가 나오게 되지만, 여기서는 해당되지 않습니다. 좀 더
다양한 패턴의 입력은 뒤의 텍스트 분류, 개체명 인식, 질의 응답 실습에서 이어서 보겠습니다.

###3. 토큰 예측하기
FillMaskPipeline 은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서
보여줍니다. FillMaskPipeline 에 우선 앞서 불러온 모델과 토크나이저를 지정해줍니다.

In [8]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Device set to use 0


이제 입력 문장으로부터 [MASK] 의 위치에 들어갈 수 있는 상위 5 개의 후보 단어들을 출력해봅시다.

In [9]:
pip('Soccer is a really fun [MASK].')

[{'score': 0.7621113061904907,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.20342056453227997,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.012208598665893078,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.001863026642240584,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.001335486420430243,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [10]:
pip('The Avengers is really fun [MASK].')

[{'score': 0.19735634326934814,
  'token': 2295,
  'token_str': 'though',
  'sequence': 'the avengers is really fun though.'},
 {'score': 0.17442849278450012,
  'token': 4933,
  'token_str': 'stuff',
  'sequence': 'the avengers is really fun stuff.'},
 {'score': 0.10920818895101547,
  'token': 2205,
  'token_str': 'too',
  'sequence': 'the avengers is really fun too.'},
 {'score': 0.02360236831009388,
  'token': 2182,
  'token_str': 'here',
  'sequence': 'the avengers is really fun here.'},
 {'score': 0.021633947268128395,
  'token': 1000,
  'token_str': '"',
  'sequence': 'the avengers is really fun ".'}]

In [11]:
pip('I went to [MASK] this morning.')

[{'score': 0.3573073446750641,
  'token': 2147,
  'token_str': 'work',
  'sequence': 'i went to work this morning.'},
 {'score': 0.23304398357868195,
  'token': 2793,
  'token_str': 'bed',
  'sequence': 'i went to bed this morning.'},
 {'score': 0.1284506916999817,
  'token': 2082,
  'token_str': 'school',
  'sequence': 'i went to school this morning.'},
 {'score': 0.06230578571557999,
  'token': 3637,
  'token_str': 'sleep',
  'sequence': 'i went to sleep this morning.'},
 {'score': 0.046952586621046066,
  'token': 2465,
  'token_str': 'class',
  'sequence': 'i went to class this morning.'}]

## 구글 BERT의 다음 문장 예측(Next Sentence Prediction)
사전 학습된 한국어 BERT 를 이용하여 다음 문장 예측을 실습해봅시다. 이번 실습을 위해서만이 아니라
앞으로 사전 학습된 BERT 를 사용할 때는 transformers 라는 패키지를 자주 사용하게 됩니다. 실습 환경
에 transformers 패키지를 설치해둡시다.

### 1. 다음 문장 예측 모델과 토크나이저
transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT 는 이미 누군가가 학습해둔 모
델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다.

In [12]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

TFBertForNextSentencePrediction.from_pretrained(‘BERT 모델 이름’) 을 넣으면 두 개의 문장이 이어
지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드합니다.
AutoTokenizer.from_pretrained(‘모델 이름’) 을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크
나이저를 로드합니다.

In [13]:
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

###2. BERT의 입력
다음 문장 예측을 위해 문맥 상으로 실제로 이어지는 두 개의 문장을 준비합니다.

In [16]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

앞서 준비한 bert‐base‐uncased 의 토크나이저를 사용하여 두 개의 문장을 정수 인코딩해봅시다.

In [17]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

In [18]:
print(encoding['input_ids'])

tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)


여기서 주의할 점은 여기서 101 과 102 는 특별 토큰이라는 점입니다. 실제로 해당 토크나이저의 [CLS] 토
큰과 [SEP] 토큰의 번호를 출력해봅시다.

In [19]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':', tokenizer.sep_token_id)

[CLS] : 101
[SEP] : 102


위의 정수 인코딩 결과를 다시 디코딩해보면 현재 입력의 구성을 확인할 수 있습니다.

In [20]:
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


BERT 에서 두 개의 문장이 입력으로 들어갈 경우에는 맨 앞에는 [CLS] 토큰이 존재하고, 첫번째 문장이 끝
나면 [SEP] 토큰, 그리고 두번째 문장이 종료되었을 때 다시 추가적으로 [SEP] 토큰이 추가됩니다. 토크나
이저로 변환된 결과에서 token_type_ids 를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 확인할 수
있습니다.

In [22]:
print(encoding['token_type_ids'])

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 58), dtype=int32)


0 이 연속적으로 등장하다가 어느 순간부터 1 이 연속적으로 등장하는데, 이는 [CLS] 토큰의 위치부터 첫
번째 문장이 끝나고나서 등장한 [SEP] 토큰까지의 위치에는 0 이 등장하고, 다음 두번째 문장부터는 1 이
등장하는 것입니다. token_type_ids 에서는 0 과 1 로 두 개의 문장을 구분하고 있습니다.

###3. 다음 문장 예측하기
이제 TFBertForNextSentencePrediction 를 통해서 다음 문장을 예측해봅시다. 모델에 입력을 넣으면, 해
당 모델은 소프트맥스 함수를 지나기 전의 값인 logits 을 리턴합니다. 해당 값을 소프트맥스 함수를 통과
시킨 후에 각 레이블에 대한 확률값을 출력해봅시다.

In [23]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)


tf.Tensor([[9.9999714e-01 2.8381855e-06]], shape=(1, 2), dtype=float32)


0 번 인덱스에 대한 확률값이 1 번 인덱스에 대한 확률값보다 훨씬 큽니다. 실질적으로 모델이 예측한 레
이블은 0 이라는 뜻입니다. 이제 두 개의 값 중 더 큰 값을 모델의 예측값으로 판단하도록 더 큰 확률값을
가진 인덱스를 리턴하도록 합니다.

In [24]:
print('최종 예측 레이블:', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블: [0]


최종 예측 레이블은 0 입니다. 이는 BERT 가 다음 문장 예측을 학습했을 당시에 실질적으로 이어지는 두
개의 문장의 레이블은 0. 이어지지 않는 두 개의 문장의 경우에는 레이블을 1 로 두고서 이진 분류로 학습
을 하였기 때문입니다. 이번에는 이어지지 않는 두 개의 문장으로 테스트해봅시다. 전체적인 과정은 이전
과 같습니다.

In [25]:
# 상 관 없 는 두 개 의 문 장
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."

encoding = tokenizer(prompt, next_sentence, return_tensors='tf')
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 : ', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 :  [1]
